## Classification of MNIST digits using CNN

In [2]:
# import libraries
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset
from sklearn.model_selection import train_test_split

# for importing dataset
import torchvision

# for getting model summary
from torchinfo import summary

# for display options

import matplotlib.pyplot as plt

from IPython import display
display.set_matplotlib_formats('svg')

import warnings
warnings.simplefilter('ignore')

# Import and process the data

In [3]:
# To download MNIST dataset to current directory:
MNIST = torchvision.datasets.MNIST(".", download=True)

In [4]:
train_data = MNIST.train_data
train_labels = MNIST.train_labels
test_data = MNIST.test_data
test_labels = MNIST.test_labels

In [ ]:
print(f'{train_data.shape}\n{train_labels.shape}\n{test_data.shape}\n{test_labels.shape}')

In [5]:
# reshaping data to 2D
train_data = train_data.reshape(test_data.shape[0],1,28,28)
test_data = test_data.reshape(test_data.shape[0],1,28,28)

# DataLoader object

In [6]:
# Converting to tensor

train_data = torch.tensor( train_data ).float()
train_labels = torch.tensor( train_labels ).long()
test_data = torch.tensor( test_data ).float()
test_labels = torch.tensor( train_labels ).long()

# Converting to PyTorch Datasets

train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)

# Creating dataloader objects

batchsize    = 32
train_loader = DataLoader(train_data,batch_size=batchsize,shuffle=True,drop_last=True)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])

In [ ]:
# check size to conform to format: images X channels X width X height
train_loader.dataset.tensors[0].shape

# Deep learning model

In [7]:
# creating a class for the model
def createTheMNISTNet(printtoggle=False):

  class mnistNet(nn.Module):
    def __init__(self,printtoggle):
      super().__init__()

      ### convolution layers
      self.conv1 = nn.Conv2d( 1,10,kernel_size=5,stride=1,padding=1)
#       Nh = floor((Mh + 2p - k)/Sh) + 1, where:
#        Nh = no of pixels in current layer, Mh = no of pixels in previous layer
#        sh = stride, p = padding, k = no of pixels in kernel, h (subscript) = height

      # size: np.floor( (28+2*1-5)/1 )+1 = 26/2 = 13 (/2 b/c maxpool)

      self.conv2 = nn.Conv2d(10,20,kernel_size=5,stride=1,padding=1)
      # size: np.floor( (13+2*1-5)/1 )+1 = 11/2 = 5 (/2 b/c maxpool)

      # computing the number of units in FClayer (number of outputs of conv2)
      expectSize = np.floor( (5+2*0-1)/1 ) + 1 # fc1 layer has no padding or kernel, so set to 0/1
      expectSize = 20*int(expectSize**2)
      
      ### fully-connected layer
      self.fc1 = nn.Linear(expectSize,50)

      ### output layer
      self.out = nn.Linear(50,10)

      # toggle for printing out tensor sizes during forward prop
      self.print = printtoggle

    # forward pass
    def forward(self,x):
      
      print(f'Input: {x.shape}') if self.print else None

      # convolution -> maxpool -> relu
      x = F.relu(F.max_pool2d(self.conv1(x),2))
      print(f'Layer conv1/pool1: {x.shape}') if self.print else None

      # convolution -> maxpool -> relu
      x = F.relu(F.max_pool2d(self.conv2(x),2))
      print(f'Layer conv2/pool2: {x.shape}') if self.print else None

      # reshape for linear layer
      nUnits = x.shape.numel()/x.shape[0]
      x = x.view(-1,int(nUnits))
      if self.print: print(f'Vectorize: {x.shape}')
      
      # linear layers
      x = F.relu(self.fc1(x))
      if self.print: print(f'Layer fc1: {x.shape}')
      x = self.out(x)
      if self.print: print(f'Layer out: {x.shape}')

      return x
  
  # creating the model instance
  net = mnistNet(printtoggle)
  
  # loss function
  lossfun = nn.CrossEntropyLoss()

  # optimizer
  optimizer = torch.optim.Adam(net.parameters(),lr=.001)

  return net,lossfun,optimizer

In [8]:
# testing the model with one batch
net,lossfun,optimizer = createTheMNISTNet(True)

X,y = next(iter(train_loader))
yHat = net(X)


# checking sizes of model outputs and target variable
print(' ')
print(yHat.shape)
print(y.shape)

# computing the loss
loss = lossfun(yHat,y)
print(' ')
print('Loss:')
print(loss)

Input: torch.Size([32, 1, 28, 28])
Layer conv1/pool1: torch.Size([32, 10, 13, 13])
Layer conv2/pool2: torch.Size([32, 20, 5, 5])
Vectorize: torch.Size([32, 500])
Layer fc1: torch.Size([32, 50])
Layer out: torch.Size([32, 10])
 
torch.Size([32, 10])
torch.Size([32])
 
Loss:
tensor(15.2364, grad_fn=<NllLossBackward0>)


In [9]:
# counting the total number of parameters in the model
summary(net,(1, 1,28,28))

Input: torch.Size([1, 1, 28, 28])
Layer conv1/pool1: torch.Size([1, 10, 13, 13])
Layer conv2/pool2: torch.Size([1, 20, 5, 5])
Vectorize: torch.Size([1, 500])
Layer fc1: torch.Size([1, 50])
Layer out: torch.Size([1, 10])


Layer (type:depth-idx)                   Output Shape              Param #
mnistNet                                 [1, 10]                   --
├─Conv2d: 1-1                            [1, 10, 26, 26]           260
├─Conv2d: 1-2                            [1, 20, 11, 11]           5,020
├─Linear: 1-3                            [1, 50]                   25,050
├─Linear: 1-4                            [1, 10]                   510
Total params: 30,840
Trainable params: 30,840
Non-trainable params: 0
Total mult-adds (M): 0.81
Input size (MB): 0.00
Forward/backward pass size (MB): 0.07
Params size (MB): 0.12
Estimated Total Size (MB): 0.20

# Create a function that trains the model

In [10]:
# a function that trains the model

def function2trainTheModel():

  # number of epochs
  numepochs = 10
  
  # creating a new model
  net,lossfun,optimizer = createTheMNISTNet()

  # initializing losses
  losses    = torch.zeros(numepochs)
  trainAcc  = []
  testAcc   = []


  # looping over epochs
  for epochi in range(numepochs):

    # looping over training data batches
    net.train()
    batchAcc  = []
    batchLoss = []
    for X,y in train_loader:

      # forward pass and loss
      yHat = net(X)
      loss = lossfun(yHat,y)

      # backprop
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # loss from this batch
      batchLoss.append(loss.item())

      # computing accuracy
      matches = torch.argmax(yHat,axis=1) == y     # booleans (false/true)
      matchesNumeric = matches.float()             # convert to numbers (0/1)
      accuracyPct = 100*torch.mean(matchesNumeric) # average and x100
      batchAcc.append( accuracyPct )               # add to list of accuracies
    # end of batch loop...

    # after trained through the batches, get average training accuracy
    trainAcc.append( np.mean(batchAcc) )

    # get average losses across the batches
    losses[epochi] = np.mean(batchLoss)

    # test accuracy
    net.eval()
    X,y = next(iter(test_loader)) # extract X,y from test dataloader
    with torch.no_grad(): # deactivates autograd
      yHat = net(X)
      
    # comparing to the training accuracy lines
    testAcc.append( 100*torch.mean((torch.argmax(yHat,axis=1)==y).float()) )

  # end epochs

  # function output
  return trainAcc,testAcc,losses,net


# Run the model and show the results

In [11]:
trainAcc,testAcc,losses,net = function2trainTheModel()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,5))

ax[0].plot(losses,'s-')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss')
ax[0].set_title('Model loss')

ax[1].plot(trainAcc,'s-',label='Train')
ax[1].plot(testAcc,'o-',label='Test')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Accuracy (%)')
ax[1].set_title(f'Final model test accuracy: {testAcc[-1]:.2f}%')
ax[1].legend()

plt.show()